In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_name = "meta-llama/Llama-3.1-8B-Instruct"  # Replace with exact path for 8B model
tokenizer = AutoTokenizer.from_pretrained(model_name)

/home/yg9bq/miniconda/ENTER/envs/llama/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,       
    output_hidden_states=True,   
    low_cpu_mem_usage=True,
    torch_dtype=torch.float16,     # Use float16 for efficient memory usage on GPU
    device_map=None,  # Automatically distribute across devices or use CPU
    trust_remote_code=True
)

# # Example sentence
# sentence = "This is an example sentence."

# # Tokenize the input sentence
# inputs = tokenizer(sentence, return_tensors='pt')

# # Forward pass through the model to get hidden states
# with torch.no_grad():
#     outputs = model(**inputs)

# # Extract the hidden states (embeddings)
# hidden_states = outputs.hidden_states  # This returns a tuple with hidden states for each layer

# # Get the final layer embeddings (usually the last layer's hidden state)
# # hidden_states[-1] gives the final layer's hidden states
# sentence_embedding = hidden_states[-1][:, 0, :]  # Usually, the [CLS] token embedding is at position 0

# # Convert to numpy or keep as tensor for further processing
# embedding = sentence_embedding.cpu().numpy()
# print("Embedding shape:", embedding.shape)


/home/yg9bq/miniconda/ENTER/envs/llama/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:777: UserWarning: `return_dict_in_generate` is NOT set to `True`, but `output_hidden_states` is. When `return_dict_in_generate` is not `True`, `output_hidden_states` is ignored.
  warnings.warn(
Loading checkpoint shards: 100%|██████████| 4/4 [00:31<00:00,  7.91s/it]
Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)


Embedding shape: (1, 4096)


In [4]:
import pandas as pd

answer_df = pd.read_csv("answer_df_new.csv")
query_df = pd.read_csv("query_df.csv")

In [30]:
text = "i hate you"
inputs = tokenizer(text, return_tensors='pt')
with torch.no_grad():
    outputs = model(**inputs)
hidden_states = outputs.hidden_states  # This returns a tuple with hidden states for each layer
hidden_states

(tensor([[[ 2.6512e-04, -4.9973e-04, -5.8365e-04,  ...,  3.8147e-03,
            6.3419e-05,  1.1902e-03],
          [ 1.4191e-03, -3.0212e-03, -2.4414e-03,  ..., -4.7913e-03,
            3.8605e-03,  7.4768e-03],
          [-1.4404e-02,  1.4526e-02, -1.1597e-02,  ..., -4.2725e-03,
           -3.1586e-03, -3.8528e-04],
          [-7.0801e-03, -5.4321e-03, -1.3275e-03,  ..., -2.1973e-03,
            1.5717e-03,  1.3504e-03]]], dtype=torch.float16),
 tensor([[[ 0.0022,  0.0041, -0.0007,  ...,  0.0190, -0.0042, -0.0026],
          [-0.0166,  0.0094,  0.0166,  ..., -0.0391,  0.0065,  0.0098],
          [-0.0106,  0.0027, -0.0029,  ..., -0.0298, -0.0054, -0.0198],
          [-0.0007, -0.0083,  0.0069,  ..., -0.0025,  0.0087,  0.0082]]],
        dtype=torch.float16),
 tensor([[[-1.0358e-01,  7.1167e-02, -7.1655e-02,  ...,  8.3691e-01,
            1.8140e-01,  1.1584e-01],
          [-6.5002e-03,  2.0065e-02,  7.7972e-03,  ..., -3.9368e-02,
           -7.5951e-03,  6.3171e-03],
          [-9.

In [38]:
text = "i love you too"
inputs = tokenizer(text, return_tensors='pt')
with torch.no_grad():
    outputs = model(**inputs)
hidden_states = outputs.hidden_states  # This returns a tuple with hidden states for each layer
hidden_states

(tensor([[[ 2.6512e-04, -4.9973e-04, -5.8365e-04,  ...,  3.8147e-03,
            6.3419e-05,  1.1902e-03],
          [ 1.4191e-03, -3.0212e-03, -2.4414e-03,  ..., -4.7913e-03,
            3.8605e-03,  7.4768e-03],
          [-6.3477e-03, -6.7139e-03, -1.5945e-03,  ..., -5.3711e-03,
           -7.0801e-03, -2.3079e-04],
          [-7.0801e-03, -5.4321e-03, -1.3275e-03,  ..., -2.1973e-03,
            1.5717e-03,  1.3504e-03],
          [-1.5442e-02, -1.7929e-04,  5.8899e-03,  ..., -5.7068e-03,
           -1.1719e-02, -6.7444e-03]]], dtype=torch.float16),
 tensor([[[ 0.0022,  0.0041, -0.0007,  ...,  0.0190, -0.0042, -0.0026],
          [-0.0166,  0.0094,  0.0166,  ..., -0.0391,  0.0065,  0.0098],
          [-0.0013,  0.0057,  0.0136,  ..., -0.0254, -0.0064, -0.0138],
          [-0.0037,  0.0010,  0.0025,  ...,  0.0082,  0.0029,  0.0141],
          [-0.0346, -0.0188, -0.0007,  ..., -0.0074,  0.0012,  0.0092]]],
        dtype=torch.float16),
 tensor([[[-1.0358e-01,  7.1167e-02, -7.1655e-02,

In [39]:
sentence_embedding = hidden_states[-1][:, :, :]  #  each word has an embedding vector
sentence_embedding.shape

torch.Size([1, 5, 4096])

In [36]:
sentence_embedding = hidden_states[-1][:, :, :]  # Usually, the [CLS] token embedding is at position 0

# print the shape of sentence_embedding

    # Convert to numpy or keep as tensor for further processing
embedding = sentence_embedding.cpu().numpy()
embedding

array([[ 1.334  ,  2.781  ,  1.642  , ..., -1.947  ,  2.754  ,  1.934  ],
       [-2.102  , -0.3083 , -2.621  , ..., -0.42   ,  1.42   ,  0.01744],
       [-0.4412 , -1.726  ,  0.363  , ..., -1.77   ,  1.033  , -0.6167 ],
       [-1.456  , -0.2412 ,  2.754  , ..., -0.1048 ,  0.3694 , -1.004  ]],
      dtype=float16)

In [29]:
def get_embeddings(text):
    # Tokenize the input sentence
    inputs = tokenizer(text, return_tensors='pt')

    # Forward pass through the model to get hidden states
    with torch.no_grad():
        outputs = model(**inputs)

    # Extract the hidden states (embeddings)
    hidden_states = outputs.hidden_states  # This returns a tuple with hidden states for each layer

    # Get the final layer embeddings (usually the last layer's hidden state)
    # hidden_states[-1] gives the final layer's hidden states
    sentence_embedding = hidden_states[-1][:, 0, :]  # Usually, the [CLS] token embedding is at position 0

    # Convert to numpy or keep as tensor for further processing
    embedding = sentence_embedding.cpu().numpy()

    return embedding[0].tolist()

# example usage :
get_embeddings("Family and social relationships")

[1.333984375,
 2.78125,
 1.6416015625,
 2.822265625,
 3.421875,
 1.892578125,
 -1.2197265625,
 -0.6943359375,
 -1.8876953125,
 1.5185546875,
 -0.8837890625,
 -0.218505859375,
 2.1484375,
 -2.9375,
 -0.88134765625,
 -1.1103515625,
 0.85302734375,
 0.98291015625,
 -2.69921875,
 -3.388671875,
 -2.541015625,
 0.94091796875,
 -3.53125,
 3.23828125,
 0.77001953125,
 -0.33984375,
 0.33984375,
 1.39453125,
 -2.09375,
 0.48095703125,
 -0.035400390625,
 -0.397705078125,
 2.55859375,
 -3.021484375,
 -3.400390625,
 -0.274658203125,
 0.11468505859375,
 -2.0234375,
 1.0048828125,
 -4.5234375,
 3.62109375,
 -2.140625,
 1.4072265625,
 1.3134765625,
 -0.99755859375,
 3.08203125,
 2.177734375,
 -0.66455078125,
 2.8671875,
 -0.2113037109375,
 -2.9921875,
 1.447265625,
 1.4169921875,
 -0.240478515625,
 -1.1865234375,
 1.2724609375,
 -1.6123046875,
 -0.830078125,
 0.58544921875,
 -1.13671875,
 1.1943359375,
 -1.552734375,
 -0.54541015625,
 -1.421875,
 3.421875,
 2.59375,
 -1.3896484375,
 -2.201171875,
 -0.

In [18]:

query_text = "i hate my body"
query_embeddings = [get_embeddings(query_text)]



In [28]:
piece_text = "i love my body"
piece_embeddings = [get_embeddings(piece_text)]


In [27]:
piece_embeddings == query_embeddings

True

In [25]:
np.array(query_embeddings[0])

array([ 1.33398438,  2.78125   ,  1.64160156, ..., -1.94726562,
        2.75390625,  1.93359375])

In [23]:
np.array(piece_embeddings[0])

array([ 1.33398438,  2.78125   ,  1.64160156, ..., -1.94726562,
        2.75390625,  1.93359375])

In [24]:
[np.dot(np.array(piece_embeddings[i]), np.array(query_embeddings[i]).T).item() for i in range(len(piece_embeddings))]


[20543.570416974533]

In [17]:
# %pip install scikit-learn
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def get_similarities(piece_text, query_text):
    piece_embeddings = [get_embeddings(piece_text)]
    query_embeddings = [get_embeddings(query_text)]
    cosine_similarities = [cosine_similarity([piece_embeddings[i]], [query_embeddings[i]])[0][0] for i in range(len(piece_embeddings))]
    inner_products = [np.dot(piece_embeddings[i], query_embeddings[i].T).item() for i in range(len(piece_embeddings))]
    
    return cosine_similarities, inner_products

get_cosine_similarities("body dissatisfaction", "i hate my body")

[1.0000000000000004]

In [ ]:
def get_cosine_similarities(piece_text, query_text):
    piece_embeddings = [get_embeddings(piece_text)]
    query_embeddings = [get_embeddings(query_text)]
    cosine_similarities = [cosine_similarity([piece_embeddings[i]], [query_embeddings[i]])[0][0] for i in range(len(piece_embeddings))]
    return cosine_similarities

def get_inner_products(piece_text, query_text):
    piece_embeddings = [get_embeddings(piece_text, model) for model in mdl_ls]
    query_embeddings = [get_embeddings(query_text, model) for model in mdl_ls]
    inner_products = [np.dot(piece_embeddings[i], query_embeddings[i].T).item() for i in range(len(piece_embeddings))]
    
    return inner_products

def process_paragraph(text):
    # Split the text by the " EOS " marker
    sentences = re.split(r'\. ', text) # split by period 
    
    # Remove any sentences with fewer than two words
    filtered_sentences = [sentence.strip() for sentence in sentences if len(sentence.split()) >= 2]
    
    return filtered_sentences

def get_paragraph_sim(text, query_text, plot=False, sim_by="MIP"):
    # MIP: maximum inner product
    # MCS: maximum cosine similarity
    
    pieces = process_paragraph(text)
    results = []  # List to hold results
    for piece_text in pieces:
        if sim_by == "MCS":
            result = get_cosine_similarities(piece_text, query_text)
        if sim_by == "MIP":
            result = get_inner_products(piece_text, query_text)
            
        results.append(result)
        
        
    results = np.array(results)
    if plot:
        plt.figure(figsize=(10, 6))
        for i in range(results.shape[1]):
            plt.plot(results[:, i], label=f'Model {i+1}')

        # Add labels and title
        plt.ylim(-1,1)
        plt.xlabel("sentence order")
        plt.ylabel("Similarity")
        plt.legend()
        plt.show()
        
    return results, pieces


In [6]:
# Example sentence
sentence = query_df.description[0]

# Tokenize the input sentence
inputs = tokenizer(sentence, return_tensors='pt')

# Forward pass through the model to get hidden states
with torch.no_grad():
    outputs = model(**inputs)

# Extract the hidden states (embeddings)
hidden_states = outputs.hidden_states  # This returns a tuple with hidden states for each layer

# Get the final layer embeddings (usually the last layer's hidden state)
# hidden_states[-1] gives the final layer's hidden states
sentence_embedding = hidden_states[-1][:, 0, :]  # Usually, the [CLS] token embedding is at position 0

# Convert to numpy or keep as tensor for further processing
embedding = sentence_embedding.cpu().numpy()
print("Embedding shape:", embedding.shape)

Embedding shape: (1, 4096)


In [10]:
embedding[0].tolist()

[1.333984375,
 2.78125,
 1.6416015625,
 2.822265625,
 3.421875,
 1.892578125,
 -1.2197265625,
 -0.6943359375,
 -1.8876953125,
 1.5185546875,
 -0.8837890625,
 -0.218505859375,
 2.1484375,
 -2.9375,
 -0.88134765625,
 -1.1103515625,
 0.85302734375,
 0.98291015625,
 -2.69921875,
 -3.388671875,
 -2.541015625,
 0.94091796875,
 -3.53125,
 3.23828125,
 0.77001953125,
 -0.33984375,
 0.33984375,
 1.39453125,
 -2.09375,
 0.48095703125,
 -0.035400390625,
 -0.397705078125,
 2.55859375,
 -3.021484375,
 -3.400390625,
 -0.274658203125,
 0.11468505859375,
 -2.0234375,
 1.0048828125,
 -4.5234375,
 3.62109375,
 -2.140625,
 1.4072265625,
 1.3134765625,
 -0.99755859375,
 3.08203125,
 2.177734375,
 -0.66455078125,
 2.8671875,
 -0.2113037109375,
 -2.9921875,
 1.447265625,
 1.4169921875,
 -0.240478515625,
 -1.1865234375,
 1.2724609375,
 -1.6123046875,
 -0.830078125,
 0.58544921875,
 -1.13671875,
 1.1943359375,
 -1.552734375,
 -0.54541015625,
 -1.421875,
 3.421875,
 2.59375,
 -1.3896484375,
 -2.201171875,
 -0.